In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [3]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [4]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [8]:
prompt = """
<human>: Json query for meetings of last 10 days
<assistant>:
""".strip()

In [9]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [10]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Json query for meetings of last 10 days
<assistant>: [
  {
    date: "2021-10-01",
    title: "Meeting 1",
    attendees: [
      { name: "John", email: "john@example.com" },
      { name: "Jane", email: "jane@example.com" }
    ]
  },
  {
    date: "2021-10-02",
    title: "Meeting 2",
    attendees: [
      { name: "John", email: "john@example.com" },
      { name: "Jane", email: "jane@example.com" }
    ]
  },
  {
    date: "2021-10-03",
    title: "Meeting 3",
    attendees: [
      { name: "John", email: "john@example.com" },
      { name:
CPU times: user 1min 14s, sys: 324 ms, total: 1min 15s
Wall time: 1min 19s


# Prep dataset

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
data = load_dataset("csv", data_files="/content/drive/MyDrive/Colab Notebooks/data22.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7c806898a1314de6/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['Text ', 'JSON '],
        num_rows: 207
    })
})

In [14]:
data["train"][0]

{'Text ': 'Total number of users',
 'JSON ': '[{"table": "users","columns": {"id": {"operation": "COUNT"}}}]'}

In [15]:
data.keys()

dict_keys(['train'])

In [16]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Text "]}
<assistant>: {data_point["JSON "]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [17]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

In [18]:
data

Dataset({
    features: ['Text ', 'JSON ', 'input_ids', 'attention_mask'],
    num_rows: 207
})

# Finetune the model

In [19]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=2,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.711300
2,2.458600
3,2.652300
4,2.555500
5,2.648700
6,2.638900
7,2.653000
8,2.342800
9,2.463000
10,2.260100


TrainOutput(global_step=102, training_loss=0.721807091113399, metrics={'train_runtime': 578.1869, 'train_samples_per_second': 0.716, 'train_steps_per_second': 0.176, 'total_flos': 635601637122816.0, 'train_loss': 0.721807091113399, 'epoch': 1.97})

# Save trained model

In [20]:
model.save_pretrained("trained-model")

In [21]:
PEFT_MODEL = "MohamedTahir/Falcon7B_text_to_json"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True,
    # create_pr=1
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:273: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:273: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/MohamedTahir/Falcon7B_text_to_json/commit/eaead6a994252c1dae8161d3e46a0461217132df', commit_message='Upload model', commit_description='', oid='eaead6a994252c1dae8161d3e46a0461217132df', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [23]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [27]:
%%time
device = "cuda:0"

prompt = """Given the following schemas: The 'users' table schema as:(id,name,email,phone_number,country,gender,timezone,avatar,is_active,access_token,refresh_token,create_date,meeting_id,is_zoom,zoom_link,meeting_preference,user_role,is_onboarded,zoom_refresh_token,city,work_place,about,language,zip,zoom_user_id,permanent_meeting_location,wake_time,primary_calendar_id,provider_id,zoom_user_email,primary_conference_id).

The "meetings" table schema as:(id,host_id,title,description,location,link,start_datetime,end_datetime,request_id,meeting_generated,created_date,type,event_id,nlp_output,calendar_id,account_user_id).
Generate a generlized json format for the following text for example input is no of meetings output is {
  "table": "meetings",
  "columns": {
    "id": {
      "operation": "COUNT"
    }
  }
}
use named entity recognition and map the given input to its respective column in the table.
another example is
input is zipcode for tejas output should be {
  "table": "users",
  "columns": {
    "zip": {
      "operation": " "
    },
    "name": {
      "value": "tejas",
      "filter": "="
    }
  }
}
Text:no of meetings on ZOOM
JSON:
""".strip()


encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Given the following schemas: The 'users' table schema as:(id,name,email,phone_number,country,gender,timezone,avatar,is_active,access_token,refresh_token,create_date,meeting_id,is_zoom,zoom_link,meeting_preference,user_role,is_onboarded,zoom_refresh_token,city,work_place,about,language,zip,zoom_user_id,permanent_meeting_location,wake_time,primary_calendar_id,provider_id,zoom_user_email,primary_conference_id).

The "meetings" table schema as:(id,host_id,title,description,location,link,start_datetime,end_datetime,request_id,meeting_generated,created_date,type,event_id,nlp_output,calendar_id,account_user_id).
Generate a generlized json format for the following text for example input is no of meetings output is {
  "table": "meetings",
  "columns": {
    "id": {
      "operation": "COUNT"
    }
  }
}
use named entity recognition and map the given input to its respective column in the table.
another example is
input is zipcode for tejas output should be {
  "table": "users",
  "columns": {
 